## Clone repository heart-segmentation-monai

In [1]:
!git clone https://github.com/luhtookyaw/heart-segmentation-monai.git

## Install Necessary packages

In [2]:
!pip3 install monai
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

## Import packages

In [3]:
import sys

sys.path.append("/content/heart-segmentation-monai/")

In [ ]:
import torch
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.losses import DiceLoss, DiceCELoss
from utilities import prepare, train

## Configurations and Training

### Define input data directory and saved model directory

In [ ]:
data_dir = "/content/heart-segmentation-monai/Task_Heart"
model_dir = "/content/heart-segmentation-monai/"

data_in = prepare(data_dir, a_min=0, a_max=1435.2, cache=True) # contrast between 0 and 1435.2

### Define CUDA gpu device

In [ ]:
device = torch.device("cuda:0")
model = UNet(
    dimensions=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256), 
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
).to(device)

### Define Dice Loss or Dice Cross Entropy Loss

In [ ]:
#loss_function = DiceCELoss(to_onehot_y=True, sigmoid=True, squared_pred=True, ce_weight=calculate_weights(1792651250,2510860).to(device))
loss_function = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-5, weight_decay=1e-5, amsgrad=True)

### Train the model

In [ ]:
train(model, data_in, loss_function, optimizer, 20, model_dir)